In [ ]:
## data visualization
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import pandas as pd
import numpy as np
import yaml
import re
import nltk
from nltk import word_tokenize
import datetime
from datetime import datetime


#pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import os
from functools import reduce
from collections import OrderedDict 

control_color = "#444444"
treatment_color = "#2B4888"

## cleaner school names
schoolmap_msg = {'Anacostia HS': "Anacostia", 
                 'Columbia Heights EC (CHEC)': "CHEC",
                 'Dunbar HS': 'Dunbar'}

# 1. Load messaging data from script 5

In [ ]:
msg_data = pd.read_pickle('../data/analysis_data/full_year_msg_data_pickle.pkl')
msg_data.shape

In [ ]:
print('N messages by school:')
pd.merge(msg_data.school_merge.value_counts().reset_index(),
         msg_data.school_merge.value_counts(normalize = True).reset_index(), 
         on = 'index').rename(columns = {'index': 'school_name',
                                         'school_merge_x': 'counts',
                                         'school_merge_y':'pct'})

print('N students per school:')
msg_data[['school_merge','StudentID']].drop_duplicates().groupby('school_merge').count()\
.reset_index().rename(columns = {'StudentID': 'count_unique_students'})

# 2. Load and prep OSSE data pulled from Qlik

In [ ]:
# osse ids for all 6 schools
osse_ids = pd.read_csv("../data/analysis_data/osse_data_formatching_withdem_all6_qlik.csv")

#osse_ids_SY1920.head()

In [ ]:
osse_ids['osse_school_merge'] = np.where(osse_ids.school_name.str.contains("Columbia Heights"),
                                'CHEC',
                           np.where(osse_ids.school_name.str.contains("Anacostia"),
                                'Anacostia', 
                           np.where(osse_ids.school_name.str.contains("Dunbar"),
                                 'Dunbar',
                           np.where(osse_ids.school_name.str.contains("Paul"),
                                 'Paul',
                           np.where(osse_ids.school_name.str.contains("Friendship"),
                                 'Friendship',
                           'Johnson'))))) 

osse_ids[['school_name', 'osse_school_merge']].drop_duplicates()

# Rename columns so we don't get duplicates leading to _x, _y on merge
osse_ids.rename(columns = {'school_name': 'osse_school_name'}, inplace=True)

In [ ]:
# subset demographics to more recent SY (might use older SY if missing in recent one)
osse_ids_SY1920 = osse_ids[osse_ids.school_year == "SY1920"].copy()
osse_ids_SY1819 = osse_ids[osse_ids.school_year == "SY1819"].copy()

osse_ids_SY1920.shape
osse_ids_SY1819.shape

# 3. Merge message data to OSSE data

## 3.1. Merge to SY1920 OSSE data first

In [ ]:
# Merge all messages to the osse info for SY1920

osse_msg_merge_1920 = pd.merge(msg_data, 
                                    osse_ids_SY1920, 
                                    how = 'left', 
                                    left_on = ['school_merge', 'StudentID'], 
                                    right_on = ['osse_school_merge', 'lea_student_id'], 
                                    indicator = 'found_osse1920')


#osse_msg_merge_1920.head()

osse_msg_merge_1920.found_osse1920.value_counts()
osse_msg_merge_1920.found_osse1920.value_counts(normalize = True)


osse_msg_merge_1920['source_of_match'] = np.where(osse_msg_merge_1920.found_osse1920 == 'both', 
                                                  'osse_sy1920', 
                                                  'unmatched')

## 3.2. For those that did not get a match for in the first round of matching to SY1920 data, merge to SY1819 data.

In [ ]:
# Try to match the remaining one not found in OSSE SY19-20 to SY18-19
left_only = osse_msg_merge_1920[osse_msg_merge_1920.found_osse1920 == 'left_only'].copy()

# keep only columns used in the orignal msg data to avoid duplicate columns
msg_data_columns = list(msg_data.columns)
left_only[msg_data_columns].shape

remain_schools_osse_merge = pd.merge(left_only[msg_data_columns],
                                     osse_ids_SY1819, 
                                     how = 'left', 
                                     left_on = ['school_merge', 'StudentID'], 
                                     right_on = ['osse_school_merge', 'lea_student_id'], 
                                     indicator = 'found_osse1819')

remain_schools_osse_merge['source_of_match'] = np.where(remain_schools_osse_merge.found_osse1819 == 'both', 
                                                        'osse_sy1819', 
                                                        'unmatched')
remain_schools_osse_merge.found_osse1819.value_counts()
remain_schools_osse_merge.found_osse1819.value_counts(normalize = True)

remain_schools_osse_merge.shape

## 3.3. Concat the matches together

In [ ]:
osse_merge_6_schools = pd.concat([osse_msg_merge_1920[osse_msg_merge_1920.source_of_match == 'osse_sy1920']\
                                  .drop(columns='found_osse1920'), 
                                 remain_schools_osse_merge.drop(columns='found_osse1819')])
osse_merge_6_schools.shape

## 3.4. Clean up

In [ ]:
# Remove the columns that are leading to duplicates 
# 'student_tokens' only bc you can't dedupe with a list in a column
osse_merge_6_schools.drop(columns = ['enrollment_date', 
                                     'withdrawal_date', 
                                     'student_tokens'], inplace = True)

osse_merge_6_schools.drop_duplicates(inplace = True)

# Compare df with dems and orig msg
osse_merge_6_schools.shape
msg_data.shape

In [ ]:
print("N dupes:", osse_merge_6_schools.id.duplicated().sum())

In [ ]:
osse_merge_6_schools.source_of_match.value_counts()
osse_merge_6_schools.source_of_match.value_counts(normalize = True)

# 4. Sanity Checks

## 4.1. Randomly sample merges from the second semester schools

In [ ]:
second_sem_schools = ['Paul', 'Friendship', 'Johnson']

second_sem_merge = osse_merge_6_schools[osse_merge_6_schools.osse_school_merge.isin(second_sem_schools)].copy()
second_sem_merge.shape

In [ ]:
# Looks like the merge actually worked

#second_sem_merge[['school_merge', 'StudentID', 'student_name', 'lea_student_id', 'first_name', 'last_name',
#                 'osse_school_name']].drop_duplicates().sample(n=20)

## 4.2. Check the unmatched ones

In [ ]:
unmatched = osse_merge_6_schools[osse_merge_6_schools.source_of_match == 'unmatched'].copy()

In [ ]:
unique_unmatched_names = unmatched[['school_merge', 'student_name', 'StudentID']].drop_duplicates()

osse_ids[osse_ids.lea_student_id.isin(unique_unmatched_names.StudentID)]

# 5.0. Write files

In [ ]:
osse_merge_6_schools.to_pickle('../data/analysis_data/messages_w_demographics_osse6_schools_pickle.pkl') 

In [ ]:
# in case you're running in R
msgs_to_modify = osse_merge_6_schools.copy()

msgs_to_modify['content'] = msgs_to_modify['content'].str.replace('\r', '')
msgs_to_modify['content_upper'] = msgs_to_modify['content_upper'].str.replace('\r', '')


msgs_to_modify.shape
msgs_to_modify.to_csv('../data/analysis_data/messages_w_demographics_osse6_schools_csv.csv', index = False)

In [ ]:
check = pd.read_csv('../data/analysis_data/messages_w_demographics_osse6_schools_csv.csv')
check.shape